To create a flask application with a frotned and backend that includes a homepage with instructions, a form to input the IP address of the remote server, a form for username and password,
and a file upload option. The backend will take the IP address and ping the remote server, the username and password will be used to ssh into the remote server and the file upload will be used
to upload a file to the remote server. The frontend will display the results of the ping, ssh, and file upload.

In [1]:
# importing the required libraries
from flask import Flask, render_template, request, redirect, url_for, flash
import paramiko
import os

In [4]:
# Defining the Flask app
app = Flask(__name__)
app.secret_key = "162001"

In [2]:
# Function to establish an SSH connection to the remote server
def ssh_connect(username, password, ip_address):
    try:
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=ip_address, username=username, password=password)
        print("Connected to %s" % ip_address)
        return ssh_client
    except Exception as e:
        flash(f"Failed to connect to {ip_address} due to {e}")
        print("Connection Failed")
        return None

In [5]:
@app.route('/')
def home():
    return render_template('home.html')

In [6]:
@app.route('/connect', methods=['POST'])
def connect():
    ip_address = request.form.get('ip_address')
    return render_template('C:/Users/gmi/OneDrive - Hewlett Packard Enterprise/Documents/Flask Applications/src/login.html', ip_address=ip_address)


In [7]:
@app.route('/authenticate', methods=['POST'])
def authenticate():
    username = request.form.get('username')
    password = request.form.get('password')
    ip_address = request.form.get('ip_address')

    ssh_client = ssh_connect(username, password, ip_address)

    if ssh_client:
        return render_template('upload.html', ip_address=ip_address)
    else:
        return redirect(url_for('home'))

In [8]:
@app.route('/upload', methods=['POST'])
def upload():
    ip_address = request.form.get('ip_address')
    file = request.files['file']

    if file:
        # Save the uploaded file to a temporary location
        file_path = os.path.join('uploads', file.filename)
        file.save(file_path)

        # Perform the file transfer to the remote server using the SSH connection
        ssh_client = ssh_connect('your_username', 'your_password', ip_address)
        if ssh_client:
            sftp = ssh_client.open_sftp()
            sftp.put(file_path, '/home/ubuntu/' + file.filename)
            sftp.close()
            os.remove(file_path)
            flash(f"File uploaded successfully to {ip_address}")

        else:
            flash("File upload failed")
    else:
        flash("Please select a file to upload")

    return render_template('upload.html', ip_address=ip_address)

In [11]:
if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [3]:
from paramiko import SSHClient
from scp import SCPClient

ssh = SSHClient()
ssh.load_system_host_keys()
ssh.connect(hostname='172.23.226.203', username='root', password='162001')

# SCPCLient takes a paramiko transport as an argument
scp = SCPClient(ssh.get_transport())

scp.put('C:/BankNote_Authentication.csv', 'BankNote_Authentication.csv')
scp.get('BankNote_Authentication.csv', 'BankNote_Authentication.csv')

# Uploading the 'test' directory with its content in the
# '/home/user/dump' remote directory
scp.put('C:/BankNote_Authentication.csv', recursive=True, remote_path='/root/test_upload')

scp.close()